In [1]:
from linear_sample_3c import *
import time
from collections import namedtuple
from read_images import *#不带模糊
#from read_images_flur import *#带模糊，输出左图，右图和模糊
import os
import copy
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-

In [2]:
model_params = namedtuple("parameters",
                          'data_size,'
                           'mini_batch_size,'
                           'learning_rate,'
                           'total_epoch_num,'
                           'outputdir,'
                           'height,'
                           'width,'
                           'channel,'
                          'cutTop,'
                          'cutBottom,'
                          'cutLeft,'
                          'cutRight,'
                          'cPointRow,'
                          'cPointCol')


In [3]:
def train_feature(params, feature_in, left_ims, right_ims, tps_base, tps_matrix, linear_interpolator,update_total):
    print_str = 'train_tps_Step：{:4} | Reonstruction loss is {:4}  | Asmooth loss is {:4}| znorm loss is {:4} | Total loss is {:4}' \
                ' |  feature_var_mean is{:4}'
    # with tf.Graph().as_default(), tf.device('/gpu: 0'):
    max_step = np.int32(params.total_epoch_num)
    learning_rate_init = np.float32(params.learning_rate)

    optimize_op = tf.train.AdamOptimizer(learning_rate_init)##第一次标准TPS
    optimize_op1 = tf.train.AdamOptimizer(learning_rate_init)##交替z
    #optimize_op1 = tf.train.GradientDescentOptimizer(learning_rate_init)##交替z
    optimize_op2 = tf.train.AdamOptimizer(2e-3)##交替A 3e-3
    #optimize_op2 = tf.train.GradientDescentOptimizer(1e-2)##交替A
    
    tps_mat = tps_matrix
    feature_next = feature_in
    xymeshgrid = np.load('xymeshgrid.npy')
    
    
    with tf.variable_scope(tf.get_variable_scope()):
        tps_base = tf.constant(tps_base, dtype=tf.float32)
        left = tf.constant(left_ims[0:params.data_size], dtype=tf.float32)
        right = tf.constant(right_ims[0:params.data_size], dtype=tf.float32)
        
        
        feature_f = tf.constant(feature_in, dtype=tf.float32)
        #feature_f = tf.placeholder(tf.float32, shape=(50, 16, 1))
        tps_weight_TRUE = tf.constant(tps_matrix, dtype=tf.float32)
        tps_weight_f = tf.constant(tps_matrix, dtype=tf.float32)
        #tps_weight_f = tf.placeholder(tf.float32, shape=(57600,16))
        compensateI = tf.Variable(-2.0, dtype=tf.float32, name='contr_val')###亮度补偿
        
        
        feature_in_base = tf.constant(feature_in, dtype=tf.float32)
        disp_base = tf.zeros([params.data_size, 288, 360, 1], dtype=tf.float32)
        
        
        feature_input = tf.Variable(feature_f, dtype=tf.float32, name='contr_val')
        tps_weight = tf.Variable(tps_weight_f, dtype=tf.float32)
        update1 = tf.assign(feature_input,feature_f)
        update2 = tf.assign(tps_weight,tps_weight_f)
        
        
        disp1 = decoder_forward1(feature_input, tps_weight, linear_interpolator.sz_params)###第一次标准TPS视差计算
        disp2 = decoder_forward2(feature_input, feature_in_base, tps_weight, disp_base, linear_interpolator.sz_params)###交替训练的视差计算
        #train_dd__ = np.load('groundtruth_z/disp_batch_invivo_d.npy')[0:50]
        #disp2 = tf.Variable(train_dd__, dtype=tf.float32)
        
        
        right_est1 = linear_interpolator.interpolate(left, disp1)###第一次标准TPS生成右图
        right_est2 = linear_interpolator.interpolate(left, disp2)###交替训练生成右图
        #loss_rec = compute_rec_loss(right_est, right, linear_interpolator.sz_params)
        loss_rec1, loss_A_smooth1, compa_sum1 = compute_rec_loss160(right_est1, right, compensateI, linear_interpolator.sz_params, tps_weight_TRUE, tps_weight, xymeshgrid, 255.0, 1.0)
        loss_rec2, loss_A_smooth2, compa_sum2 = compute_rec_loss160(right_est2, right, compensateI, linear_interpolator.sz_params, tps_weight_TRUE, tps_weight, xymeshgrid, 255.0, 1.0)
        loss_rec3, loss_A_smooth3,loss_rec3_img, compa_sum3 = compute_rec_loss161(right_est2, right, compensateI, linear_interpolator.sz_params, tps_weight_TRUE, tps_weight, xymeshgrid, 255.0, 1.0)
        #loss_rec, loss_wt_norm, loss_rec_bijiao, loss_rec_batch = compute_rec_loss4(right_est, right, left, disp, linear_interpolator.sz_params)##使用膨胀做部分平滑
        #loss_rec = compute_rec_loss3(right_est, right, left, linear_interpolator.sz_params)
        
        #loss_wt_norm = tf.reduce_sum(tf.square(tps_weight - tps_base))
        
        #loss = tf.add(loss_rec, loss_wt_norm, name='Total_loss')
        #loss = tf.add(loss_rec, tf.multiply(alpha, loss_wt_norm), name='Total_loss0')
        loss1 = tf.add(loss_rec1, loss_A_smooth1, name='Total_loss1')

        #########计算是否有点里的太远影响了大局 AZ HE AZ
        disp_True = tf.zeros([params.data_size, 288, 360, 1], dtype=tf.float32)
        disp_vec1 = tf.map_fn(lambda x: tf.matmul(tps_weight, x), feature_input)#AZ
        disp_vec2 = tf.slice(disp_True, [0,  params.cutTop, params.cutLeft, 0], [-1, 200, 200, -1])
        disp_vec2 = tf.reshape(disp_vec2, [params.data_size, 200*200, 1])
        #loss_wt_norm = tf.multiply(0.02, tf.reduce_mean(tf.reduce_sum(tf.square(disp_vec1 - disp_vec2),axis = [1,2])), name='punishment')
        loss_wt_norm = 0.01*tf.reduce_mean(tf.reduce_mean(tf.square(disp_vec1 - disp_vec2),axis = [1,2]))
        #0.01
        
        #loss_wt_norm2 = tf.reduce_sum(tf.square(tps_weight - tps_weight_TRUE))
        
        loss3 = loss_rec2 + loss_A_smooth2 + loss_wt_norm#####A损失函数 4.8 40
        
        loss4 = loss_rec3
        
        
        ###优化器设置
        train_op = optimize_op.minimize(loss1,var_list = feature_input)
        train_op1 = optimize_op1.minimize(loss4,var_list = feature_input)#可以指定哪些参数进行迭代 loss_rec2
        train_op2 = optimize_op2.minimize(loss3,var_list = tps_weight)
    
    config = tf.ConfigProto(allow_soft_placement=True)
    print(1)
    with tf.Session(config=config) as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        
        
        print(1.1)
        start_time = time.time()
        loss_rec_temp = 0.
        print(1.2)
        
        feature_before = feature_in
        feature_val = 0.
        tps_before = tps_matrix
        tps_val = 0.
        disp_val = np.zeros([params.data_size, params.height, params.width, 1])
        est_right_val = np.zeros([params.data_size, params.height, params.width, params.channel])
        step = 0
        print(2)
        
        
        train_dd = np.load('output/guijiao/disp_batch_invivo_d.npy')####600训练集的d
        train_zz = np.load('output/guijiao/disp_batch_invivo_z.npy')####600训练集的z
        
        feature_base_ = np.load('output/guijiao/z_batch_invivo_mean.npy')#16*1###分别是平均的d和z
        feature_base_ =  np.tile(feature_base_,(params.data_size, 1, 1))
        
        disp_base_val = np.load('output/guijiao/disp_batch_invivo_mean.npy')
        disp_base_val =  np.tile(disp_base_val,(params.data_size, 1, 1, 1))
        
        
        out_dd = train_dd.copy()
        tps_dd = train_dd.copy()
        out_zz = train_zz.copy()
        out_left = left_ims.copy()
        out_right = right_ims.copy()
        
        totostep = 0
        inner_step = int(200/params.data_size)
        loss_result = []###记录训练过程的损失
        total_loss_result = []
        index_training = [i for i in range(len(train_zz))]
        for outstep in range(update_total):#update_total
            ##训练A
            print("训练A")
            start_time = time.time()
            
            flag = 0####过拟合标志
            
            for step in range(0, max_step):#max_step
                loss_rec_val_mean = 0
                loss_A_smooth_val_mean = 0
                loss_norm_val_mean = 0
                loss_val_mean = 0
                compa_sum = 0
                
                
                np.random.shuffle(index_training) # 打乱索引
                train_dd = train_dd[index_training]
                train_zz = train_zz[index_training]
                left_ims = left_ims[index_training]
                right_ims = right_ims[index_training]
                
                for j in range(inner_step):
                    cur_zz = train_zz[j*params.data_size:(j+1)*params.data_size]
                    cur_dd = train_dd[j*params.data_size:(j+1)*params.data_size]
                    cur_left = left_ims[j*params.data_size:(j+1)*params.data_size]
                    cur_right = right_ims[j*params.data_size:(j+1)*params.data_size]
                    
                
                    _, feature_val, tps_val, loss_rec_val,loss_A_smooth_val,loss_norm_val,compa_sum2_val, loss_val, disp_val, est_right_val = sess.run(
                        [train_op2, feature_input, tps_weight, loss_rec2,loss_A_smooth2,loss_wt_norm,compa_sum2, loss3, disp2, right_est2],
                    feed_dict={feature_input:cur_zz, tps_weight_f:tps_mat,left:cur_left, right:cur_right, 
                               feature_in_base:feature_base_, disp_base:disp_base_val, disp_True:cur_dd})

                    tps_mat = tps_val
                for j in range(inner_step):
                    cur_zz = train_zz[j*params.data_size:(j+1)*params.data_size]
                    cur_dd = train_dd[j*params.data_size:(j+1)*params.data_size]
                    cur_left = left_ims[j*params.data_size:(j+1)*params.data_size]
                    cur_right = right_ims[j*params.data_size:(j+1)*params.data_size]
                    
                    feature_val, tps_val, loss_rec_val,loss_A_smooth_val,loss_norm_val,compa_sum3_val, disp_val, est_right_val = sess.run(
                        [feature_input, tps_weight, loss_rec3_img,loss_A_smooth3,loss_wt_norm,compa_sum3, disp2, right_est2],
                    feed_dict={feature_input:cur_zz, tps_weight_f:tps_mat,left:cur_left, right:cur_right, 
                               feature_in_base:feature_base_, disp_base:disp_base_val, disp_True:cur_dd})
                    
                    loss_rec_val_mean += loss_rec_val*20
                    loss_A_smooth_val_mean += loss_A_smooth_val/inner_step
                    loss_norm_val_mean += loss_norm_val/inner_step
                    #loss_val_mean += loss_val/inner_step
                    compa_sum += compa_sum3_val
                    #print(loss_rec_val, loss_A_smooth_val)
                    #print(np.sum(compa_sum_val))
                    #print(np.sum(cur_left))
                    #if step == 0 and j == 1:
                    #    np.save('output/j1A.npy', tps_mat)
                    
                #print(loss_rec_val_mean,compa_sum)
                loss_rec_val_mean = loss_rec_val_mean/compa_sum#####计算所有误差除所有mask
                loss_val_mean = loss_rec_val_mean + loss_A_smooth_val_mean + loss_norm_val_mean
                
                loss_result.append([loss_rec_val_mean,0])#####记录A损失
                total_loss_result.append([loss_val_mean,0])
                print(loss_rec_val_mean, loss_val_mean, loss_A_smooth_val_mean, loss_norm_val_mean)
                if loss_val_mean - loss_rec_temp>0:
                    flag += 1
                else: flag = 0
                if flag>4:
                    print(3333333333)
                    print(loss_val_mean, loss_rec_temp)
                    break
                loss_rec_var = np.abs(loss_val_mean - loss_rec_temp)
                loss_rec_temp = loss_val_mean
                #if loss_rec_var < 1e-4 and step >= 50:
                #    print(step, 111111111111111)
                #    break
                if step>0 and (0 == step % 10 or step + 1 == max_step):
                #if (0 == step % 10 or step + 1 == max_step):
                    #loss_result.append([loss_rec_val_mean,0])#####记录A损失
                    print('亮度补偿', sess.run(compensateI))
                    tps_var_mean = np.mean(tps_val - tps_before)
                    feature_var_mean = np.mean(feature_val - feature_before)
                    
                    
                    print(print_str.format(totostep+step, loss_rec_val_mean,loss_A_smooth_val_mean, loss_norm_val_mean,loss_val_mean, feature_var_mean))#当前组的loss信息
                    tps_before = tps_val
                    feature_before = feature_val

                    #loss_rec_var = np.abs(loss_rec_val_mean - loss_rec_temp)
                    #loss_rec_temp = loss_rec_val_mean
                    #loss_rec_var = np.abs(loss_val_mean - loss_rec_temp)
                    #loss_rec_temp = loss_val_mean
                    #if loss_rec_var < 1e-2 or step>=100:#######考虑使用loss，比较一下和重建误差的区别
                    if step >=100 :#######考虑使用loss，比较一下和重建误差的区别
                        print(22222)
                        break
                        
            feature_val, tps_val = sess.run(
                        [feature_input, tps_weight],
                    feed_dict={feature_f:feature_next, tps_weight_f:tps_mat,left:cur_left, right:cur_right,
                               feature_in_base:feature_base_, disp_base:disp_base_val, disp_True:cur_dd})
            tps_mat = tps_val

            if feature_val.any() == feature_next.any():
                print("A结束")
            #tps_mat = tps_val####A
            print('time spent {:8} '.format(time.time() - start_time))
            distance = np.mean(np.square(feature_next - feature_base_))
            print('初始与终值的距离为:'+str(distance))
            totostep+=step
            start_time = time.time()
            
            
            
            #np.save('temp_A.npy', tps_mat)
            loss_result.append([1,1])###进入z的标识符
            total_loss_result.append([1,1])
            for j in range(inner_step):
                loss_result.append([])
                total_loss_result.append([])
                cur_zz = out_zz[j*params.data_size:(j+1)*params.data_size]
                cur_dd = tps_dd[j*params.data_size:(j+1)*params.data_size]
                cur_left = out_left[j*params.data_size:(j+1)*params.data_size]
                cur_right = out_right[j*params.data_size:(j+1)*params.data_size]
                
                
                #feature_val, tps_val, loss_rec_val,loss_A_smooth_val, loss_norm_val,loss_val, disp_val, est_right_val = sess.run(
                #        [feature_input, tps_weight, loss_rec2,loss_A_smooth2, loss_wt_norm,loss4, disp2, right_est2],
                #        feed_dict={feature_input:cur_zz,tps_weight_f:tps_mat,left:cur_left, 
                #                   right:cur_right,feature_in_base:feature_base_, disp_base:disp_base_val, disp_True:cur_dd})
                #print(loss_rec_val,loss_A_smooth_val,loss_norm_val,loss_val)
                
                sess.run(update1,feed_dict={feature_f:cur_zz})###更新futurein的值
                sess.run(update2,feed_dict={tps_weight_f:tps_mat})###更新futurein的值
                ######训练z
                print("训练z")
                for step in range(0, max_step):
                    _, feature_val, tps_val, loss_rec_val,loss_A_smooth_val, loss_norm_val,compa_sum3_val, loss_val, disp_val, est_right_val = sess.run(
                        [train_op1, feature_input, tps_weight, loss_rec3_img,loss_A_smooth3, loss_wt_norm,compa_sum3, loss4, disp2, right_est2],
                        feed_dict={tps_weight_f:tps_mat,left:cur_left, right:cur_right,
                                   feature_in_base:feature_base_, disp_base:disp_base_val, disp_True:cur_dd})
                    
                    #print(np.mean(tps_val-tps_matrix))
                    feature_next = feature_val
                    loss_result[-1].append([copy.deepcopy(loss_rec_val)*20,copy.deepcopy(compa_sum3_val)])###记录z损失
                    total_loss_result[-1].append([copy.deepcopy(loss_rec_val)*20,copy.deepcopy(compa_sum3_val), loss_norm_val+loss_A_smooth_val])
                    if  0 == step % 10 or step + 1 == max_step:
                        print('亮度补偿', sess.run(compensateI))
                        tps_var_mean = np.mean(tps_val - tps_before)
                        feature_var_mean = np.mean(feature_val - feature_before)

                        print(print_str.format(step, 20*loss_rec_val/compa_sum3_val,loss_A_smooth_val,loss_norm_val, loss_val, feature_var_mean))#当前组的loss信息
                        tps_before = tps_val
                        feature_before = feature_val

                        loss_rec_var = np.abs(loss_rec_val - loss_rec_temp)
                        loss_rec_temp = loss_rec_val
                        
                        #loss_result[-1].append([copy.deepcopy(loss_rec_val)*20,copy.deepcopy(compa_sum2_val)])###
                        #print(loss_result[j])
                        #if loss_rec_var < 1e-2 or step>=40:#######考虑使用loss，比较一下和重建误差的区别
                        if step>=50:
                            feature_val, tps_val, loss_rec_val,loss_A_smooth_val, loss_norm_val,loss_val, disp_val, est_right_val = sess.run(
                        [feature_input, tps_weight, loss_rec3_img,loss_A_smooth3, loss_wt_norm,loss4, disp2, right_est2],
                        feed_dict={feature_f:feature_next,tps_weight_f:tps_mat,left:cur_left, right:cur_right,
                                   feature_in_base:feature_base_, disp_base:disp_base_val, disp_True:cur_dd})
                            feature_next = feature_val
                            loss_rec_temp = loss_rec_val
                            
                            out_zz[j*params.data_size:(j+1)*params.data_size] = feature_val###更新train_zz的值
                            out_dd[j*params.data_size:(j+1)*params.data_size] = disp_val
                            break
                            
                
                feature = feature_val####z
                if tps_val.any() == tps_mat.any():
                    print("z结束")
                tps_mat = tps_val
                print('time spent {:8} '.format(time.time() - start_time))
                distance = np.mean(np.square(feature - feature_in))
                print('初始与终值的距离为:'+str(distance))
                
                train_dd = tps_dd.copy()
                train_zz = out_zz.copy()
                left_ims = out_left.copy()
                right_ims = out_right.copy()
                  
                start_time = time.time()
            
            
            
                #totostep+=step
            
            
             
        
        np.save('output/guijiao/jiaoti_loss.npy', loss_result)
        np.save('output/guijiao/jiaoti_total_loss.npy', total_loss_result)
        np.save('output/guijiao/out_dd.npy', out_dd)
        return feature, tps_mat, disp_val, est_right_val, distance, loss_rec_val
    



def train_z(params, feature_in, tps_base0, left_ims, right_ims, alpha, result_path,iii, update_total):
    with tf.Graph().as_default(), tf.device('/gpu: 0'):
        linear_interpolator = LinearInterpolator(params)  # initialize linear interpolator
        tps_matrix = tps_base0
        loss_seq = np.empty(shape=[0, 3], dtype=np.float32)
        print('--------------------FEATURE---------------------')
        feature_in, tps_matrix,  disp, est_right, distance, loss_rec_val= train_feature(params, feature_in, left_ims, right_ims, tps_base0,
                                                                           tps_matrix, linear_interpolator,update_total)
        
        #np.save(os.path.join(result_path, 'disp_batch_invivo20_'+str(iii)+'.npy'), disp)
        #np.save(os.path.join(result_path, 'est_right_invivo20'+str(iii)+'.npy'), est_right)
        np.save(os.path.join(result_path, 'tps_trained_.npy'), tps_matrix)
        #np.save(os.path.join(result_path, 'feature_trained_'+str(iii)+'.npy'), feature_in)
        #np.save(os.path.join(result_path, 'feature_trained_invivo'+str(iii)+'.npy'), feature_in)
        #np.save(os.path.join(result_path, 'feature_trained_laststep.npy'), feature_in)
        #np.save(os.path.join(result_path, 'loss_invivo20'+str(iii)+'.npy'), loss_rec_batch_val)
        return loss_seq

In [4]:
#通过83.2训练出来的结果 4*5
if __name__ == '__main__':
    params = model_params(data_size=20,
                              mini_batch_size=20,
                              learning_rate=3e-3,
                              total_epoch_num=np.int32(4000),
                              outputdir=r'output',
                              height=288,
                              width=360,
                              channel=3,
      
                              #cutTop=40,
                              #cutBottom=0,
                              #cutLeft=0,
                              #cutRight=90,##248 270
                          
                              cutTop=54,
                              cutBottom=34,
                              cutLeft=73,
                              cutRight=87,#######硅胶
                              
                              cPointRow=3,
                              cPointCol=3
                              )
    tps_base0 = np.loadtxt(r'A_val_real33.txt').astype(np.float32)
    #tps_base0 = np.loadtxt(r'TPS_matrix_constant.txt').astype(np.float32)
    #tps_base0 = np.load(r'tps_standard_248_270_4_5_inner.npy')
    #ids = range(1950,2000)
    #feature_in = 83.2 * np.ones([params.data_size, params.cPointRow * params.cPointCol, 1], dtype=np.float32)
    feature_in = 46.4 * np.ones([params.data_size, params.cPointRow * params.cPointCol, 1], dtype=np.float32)###硅胶
    #feature_in = 83.2 * np.random.normal(size=(params.data_size, params.cPointRow * params.cPointCol, 1))
    
    #feature_in = np.load('groundtruth_z/'+'feature_trained_final'+'.npy')[0:50,:,:]
    #feature_in0 = np.load('groundtruth_z/feature_test.npy')
    #feature_in0 = np.load('groundtruth_z/'+'feature_trained_final'+'.npy')
    #print(feature_in0.shape)

    source_img_path = '/home/ubuntu/jupyter_workspace/dataset/guijiaoheart/phantom1_rect/'
    #source_img_path = '/home/ubuntu/jupyter_workspace/dataset/real_heart/invivo1_rect/'
    #left_ims, right_ims = read_stereo_images(source_img_path, ids)
    #train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',1)
    
    loss = []
    #alpha = 0.0001
    alpha = 1.0
    for i in range(0,1):
        ids  = range(0,200)
        left_ims, right_ims = read_stereo_images(source_img_path, ids)
        left_ims = np.array(left_ims, dtype = np.float32)
        right_ims = np.array(right_ims, dtype = np.float32)
        #loss.append(train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',i))
        
        train_z(params, feature_in, tps_base0, left_ims, right_ims, alpha,'output/guijiao/',i, update_total=3)
        
        
        

Tensor("meshgrid/mul:0", shape=(288, 360), dtype=float32, device=/device:GPU:0) Tensor("meshgrid/mul_1:0", shape=(288, 360), dtype=float32, device=/device:GPU:0)
Tensor("Reshape:0", shape=(1, 103680), dtype=float32, device=/device:GPU:0) Tensor("Reshape_1:0", shape=(1, 103680), dtype=float32, device=/device:GPU:0)
Tensor("Tile:0", shape=(20, 103680), dtype=float32, device=/device:GPU:0) Tensor("Tile_1:0", shape=(20, 103680), dtype=float32, device=/device:GPU:0)
Tensor("Reshape_2:0", shape=(2073600,), dtype=float32, device=/device:GPU:0) Tensor("Reshape_3:0", shape=(2073600,), dtype=float32, device=/device:GPU:0)
--------------------FEATURE---------------------
Tensor("map/TensorArrayStack/TensorArrayGatherV3:0", shape=(20, 40000, 1), dtype=float32)
Tensor("sub_2:0", shape=(20, 9, 1), dtype=float32, device=/device:GPU:0)
Tensor("map_1/TensorArrayStack/TensorArrayGatherV3:0", shape=(20, 40000, 1), dtype=float32)
Tensor("Add:0", shape=(20, 288, 360, 1), dtype=float32, device=/device:GPU:0

66.72359625 66.9586320542866 0.20997537672519687 0.025060427561402324
66.72049333333334 66.9542581205379 0.20832444727420804 0.02544033993035555
66.717115 66.94694828718693 0.2040020376443863 0.02583124954253435
66.71261 66.94833269957512 0.2094359844923019 0.02628671508282423
66.71172708333333 66.95267990534057 0.21441456675529486 0.026538255251944063
66.70820833333333 66.95185349472364 0.21676898002624512 0.026876181364059448
66.70408791666667 66.95753646427582 0.22617973387241364 0.02726881373673677
66.70150625 66.9573745925188 0.22827753424644467 0.027590808272361753
66.69935375 66.9478244075051 0.22057615220546722 0.027894505299627788
66.69813416666666 66.94461827940623 0.21834458410739893 0.028139528632164
亮度补偿 -2.0
train_tps_Step：  70 | Reonstruction loss is 66.69813416666666  | Asmooth loss is 0.21834458410739893| znorm loss is 0.028139528632164 | Total loss is 66.94461827940623 |  feature_var_mean is0.3359881341457367
66.69385375 66.9494006653449 0.22701142728328708 0.02853548

亮度补偿 -2.0
train_tps_Step：  10 | Reonstruction loss is 68.94004583333333  | Asmooth loss is 0.29308614134788513| znorm loss is 0.03558516129851341 | Total loss is 68.94004821777344 |  feature_var_mean is-0.00045225356006994843
亮度补偿 -2.0
train_tps_Step：  20 | Reonstruction loss is 68.9169875  | Asmooth loss is 0.29308614134788513| znorm loss is 0.03564535453915596 | Total loss is 68.9169921875 |  feature_var_mean is-0.0023487939033657312
亮度补偿 -2.0
train_tps_Step：  30 | Reonstruction loss is 68.904925  | Asmooth loss is 0.29308614134788513| znorm loss is 0.035682473331689835 | Total loss is 68.90492248535156 |  feature_var_mean is-0.0007010778062976897
亮度补偿 -2.0
train_tps_Step：  40 | Reonstruction loss is 68.896825  | Asmooth loss is 0.29308614134788513| znorm loss is 0.03570728749036789 | Total loss is 68.8968276977539 |  feature_var_mean is0.0006930881063453853
亮度补偿 -2.0
train_tps_Step：  50 | Reonstruction loss is 68.891025  | Asmooth loss is 0.29308614134788513| znorm loss is 0.0357327

亮度补偿 -2.0
train_tps_Step：  10 | Reonstruction loss is 68.04804166666666  | Asmooth loss is 0.29308614134788513| znorm loss is 0.03495143726468086 | Total loss is 68.04804229736328 |  feature_var_mean is-0.0013306300388649106
亮度补偿 -2.0
train_tps_Step：  20 | Reonstruction loss is 68.02954583333333  | Asmooth loss is 0.29308614134788513| znorm loss is 0.03498833253979683 | Total loss is 68.02954864501953 |  feature_var_mean is-0.003916676621884108
亮度补偿 -2.0
train_tps_Step：  30 | Reonstruction loss is 68.02009166666667  | Asmooth loss is 0.29308614134788513| znorm loss is 0.03501753509044647 | Total loss is 68.02009582519531 |  feature_var_mean is-0.0025832282844930887
亮度补偿 -2.0
train_tps_Step：  40 | Reonstruction loss is 68.01437916666667  | Asmooth loss is 0.29308614134788513| znorm loss is 0.03504142537713051 | Total loss is 68.0143814086914 |  feature_var_mean is-0.00046168433618731797
亮度补偿 -2.0
train_tps_Step：  50 | Reonstruction loss is 68.00974583333333  | Asmooth loss is 0.29308614

66.36963666666666 66.78811936181405 0.3710407614707946 0.047441933676600455
66.36726083333333 66.7933925046279 0.378555178642273 0.04757649265229702
66.36592833333333 66.80560133167226 0.39204618334770197 0.04762681499123573
66.36637416666666 66.7979075949482 0.3839035034179688 0.04762992486357689
66.36401166666667 66.79089883055906 0.37920656800270075 0.04768059588968754
66.36183583333333 66.78212140134711 0.37254974246025085 0.04773582555353642
亮度补偿 -2.0
train_tps_Step： 180 | Reonstruction loss is 66.36183583333333  | Asmooth loss is 0.37254974246025085| znorm loss is 0.04773582555353642 | Total loss is 66.78212140134711 |  feature_var_mean is-0.038419850170612335
66.36238041666667 66.78218750802199 0.37211605906486517 0.04769103229045868
66.3604475 66.79407169381739 0.3858846724033355 0.047739521414041516
66.35936041666666 66.80350131175021 0.39643090963363653 0.04770998544991016
66.35907291666666 66.79942509614924 0.3927913904190063 0.04756078906357289
66.35708833333334 66.78977080

亮度补偿 -2.0
train_tps_Step：  10 | Reonstruction loss is 67.6380875  | Asmooth loss is 0.3926411271095276| znorm loss is 0.04709304869174957 | Total loss is 67.63807678222656 |  feature_var_mean is-0.0017836677143350244
亮度补偿 -2.0
train_tps_Step：  20 | Reonstruction loss is 67.62120833333333  | Asmooth loss is 0.3926411271095276| znorm loss is 0.04716288298368454 | Total loss is 67.62120819091797 |  feature_var_mean is-0.0037241193931549788
亮度补偿 -2.0
train_tps_Step：  30 | Reonstruction loss is 67.60865833333334  | Asmooth loss is 0.3926411271095276| znorm loss is 0.04721975699067116 | Total loss is 67.60865783691406 |  feature_var_mean is-0.0024642308708280325
亮度补偿 -2.0
train_tps_Step：  40 | Reonstruction loss is 67.598575  | Asmooth loss is 0.3926411271095276| znorm loss is 0.04726972058415413 | Total loss is 67.59857177734375 |  feature_var_mean is-0.0006317562656477094
亮度补偿 -2.0
train_tps_Step：  50 | Reonstruction loss is 67.5904125  | Asmooth loss is 0.3926411271095276| znorm loss is 0

66.25371791666667 66.69959383404176 0.39720976352691656 0.04866615384817123
66.2468525 66.69845097214818 0.40277919173240656 0.04881928041577339
66.24214916666666 66.70010107671538 0.4088999330997467 0.04905197694897652
66.23739416666666 66.7012695058469 0.41475450992584223 0.04912082925438881
66.2325675 66.6972890615958 0.4153995215892792 0.04932204000651836
66.22719416666666 66.69011395919958 0.4133626520633698 0.04955714046955108
66.225465 66.68413380789399 0.40909203886985773 0.04957676902413369
66.22155375 66.69829489145339 0.42711466550827026 0.04962647594511509
66.21752583333334 66.68608545386276 0.4189122915267945 0.04964732900261879
亮度补偿 -2.0
train_tps_Step： 210 | Reonstruction loss is 66.21752583333334  | Asmooth loss is 0.4189122915267945| znorm loss is 0.04964732900261879 | Total loss is 66.68608545386276 |  feature_var_mean is0.35173675417900085
66.21302333333334 66.68157674901923 0.41866245865821833 0.049890957027673724
66.21052916666666 66.67813979035417 0.41772866249084

66.08470166666666 66.5921989992843 0.4589613676071168 0.04853596501052379
亮度补偿 -2.0
train_tps_Step： 290 | Reonstruction loss is 66.08470166666666  | Asmooth loss is 0.4589613676071168| znorm loss is 0.04853596501052379 | Total loss is 66.5921989992843 |  feature_var_mean is-0.07764223217964172
66.08236916666667 66.61262237472276 0.4817147850990295 0.048538422957062725
66.08198833333333 66.62099952453752 0.4905115067958832 0.04849968440830708
66.0808225 66.60975482963978 0.4803711771965026 0.04856115244328976
66.07931791666667 66.6007669356815 0.4730181694030761 0.04843084961175919
66.08004625 66.58980515941917 0.46137976646423334 0.04837914295494556
66.07755291666666 66.59516893867234 0.4692718088626862 0.04834421314299107
66.078305 66.61054465987563 0.4840216636657714 0.04821799620985985
66.07503541666667 66.6119864094595 0.48857089877128607 0.04838009402155876
66.07433833333333 66.60869209433575 0.48597452044487005 0.048379240557551384
66.07397208333333 66.60790055848976 0.4856361150

亮度补偿 -2.0
train_tps_Step：  10 | Reonstruction loss is 64.68796666666667  | Asmooth loss is 0.4856361150741577| znorm loss is 0.04823252186179161 | Total loss is 64.6879653930664 |  feature_var_mean is-0.00034843021421693265
亮度补偿 -2.0
train_tps_Step：  20 | Reonstruction loss is 64.666925  | Asmooth loss is 0.4856361150741577| znorm loss is 0.04832349345088005 | Total loss is 64.66691589355469 |  feature_var_mean is-0.00091749825514853
亮度补偿 -2.0
train_tps_Step：  30 | Reonstruction loss is 64.64915833333333  | Asmooth loss is 0.4856361150741577| znorm loss is 0.048404205590486526 | Total loss is 64.64915466308594 |  feature_var_mean is0.00017507340817246586
亮度补偿 -2.0
train_tps_Step：  40 | Reonstruction loss is 64.63504166666667  | Asmooth loss is 0.4856361150741577| znorm loss is 0.04848212003707886 | Total loss is 64.6350326538086 |  feature_var_mean is5.56521954422351e-05
亮度补偿 -2.0
train_tps_Step：  50 | Reonstruction loss is 64.62335416666667  | Asmooth loss is 0.4856361150741577| znorm

/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [ ]:
减小L2增大L3 没用
增大A的步长，能够让z有用

过拟合 减少步长

不受正则化的制约，减小步长，增大正则化项  增大步长，减少正则项

In [1]:
116.29519653320312 
98.01 67.89 82.95 11752350 11779230
93.88 101.09 65.70 69.20 82.47 5818998.0 5845221.0(11554839) 5878494.0 5870157.0(116376840)

1204588800.0 1240991000.0

SyntaxError: invalid syntax (<ipython-input-1-b36e2d10f1c7>, line 2)

In [3]:
print(1204588800.0+1240991000.0)

2445579800.0


In [ ]:
import numpy as np

a = np.load(r'tps_standard_248_270_4_5_inner.npy')
b = np.loadtxt(r'TPS_matrix_constant.txt').astype(np.float32)
print(a.shape)
print(b.shape)

SyntaxError: can't assign to operator (<ipython-input-3-c6133eff9a00>, line 2)

In [2]:
##制作取点向量用于tf.gather，第三个损失函数
import tensorflow as tf
import numpy as np

def point_meshgrid(n):
    w = 240
    h = 240
    control_num_u = 4
    control_num_v = 4
    
    step_u = tf.cast(tf.floor(w / (control_num_u + 1)), tf.int32)####将一行分成n+1等分
    step_v = tf.cast(tf.floor(h / (control_num_v + 1)), tf.int32)

    edge_u_left = tf.cast(tf.floor(w % (control_num_u + 1) / 2), tf.int32)
    edge_v_up = tf.cast(tf.floor(h % (control_num_v + 1) / 2), tf.int32)
    edge_u_right = w % (control_num_u + 1) - edge_u_left
    edge_v_low = h % (control_num_v + 1) - edge_v_up

    range_start_u = edge_u_left + step_u
    range_start_v = edge_v_up + step_v
    range_end_u = w - edge_u_right - step_u
    range_end_v = h - edge_v_low - step_v

    control_u = tf.range(range_start_u, range_end_u + 1, delta=step_u)
    control_v = tf.range(range_start_v, range_end_v + 1, delta=step_v)

    c_u, c_v = tf.meshgrid(control_v, control_u)##扩展为栅格
    
    shape = tf.shape(c_u)
    a = tf.zeros(shape,dtype = 'int32')
    b = n*tf.ones(shape,dtype = 'int32')
    
    point_meshgrid_ = tf.stack([a,c_u,c_v,b],axis = 2)
    print(point_meshgrid_)
    point_meshgrid_ = tf.reshape(point_meshgrid_, [16,4])
    print(point_meshgrid_)

    return point_meshgrid_




res = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    

    
    
    for i in range(16):
        point_meshgrid_ = sess.run(point_meshgrid(i))
        res.append(point_meshgrid_)
    res = np.array(res)
    print(res.shape)
    res = res.reshape(-1,4)
    print(res[0])
    
    np.save('xymeshgrid.npy', res)
    
    tps_base0 = np.loadtxt(r'A_val_real.txt').astype(np.float32)
    aa = tf.constant(tps_base0)
    aa = tf.reshape(aa, [1,240,240,16])
    bb = tf.gather_nd(aa,res)

    print(sess.run(aa)[0,:,:,0])
    print(sess.run(bb))
    

Tensor("stack_16:0", shape=(4, 4, 4), dtype=int32)
Tensor("Reshape_17:0", shape=(16, 4), dtype=int32)
Tensor("stack_17:0", shape=(4, 4, 4), dtype=int32)
Tensor("Reshape_18:0", shape=(16, 4), dtype=int32)
Tensor("stack_18:0", shape=(4, 4, 4), dtype=int32)
Tensor("Reshape_19:0", shape=(16, 4), dtype=int32)
Tensor("stack_19:0", shape=(4, 4, 4), dtype=int32)
Tensor("Reshape_20:0", shape=(16, 4), dtype=int32)
Tensor("stack_20:0", shape=(4, 4, 4), dtype=int32)
Tensor("Reshape_21:0", shape=(16, 4), dtype=int32)
Tensor("stack_21:0", shape=(4, 4, 4), dtype=int32)
Tensor("Reshape_22:0", shape=(16, 4), dtype=int32)
Tensor("stack_22:0", shape=(4, 4, 4), dtype=int32)
Tensor("Reshape_23:0", shape=(16, 4), dtype=int32)
Tensor("stack_23:0", shape=(4, 4, 4), dtype=int32)
Tensor("Reshape_24:0", shape=(16, 4), dtype=int32)
Tensor("stack_24:0", shape=(4, 4, 4), dtype=int32)
Tensor("Reshape_25:0", shape=(16, 4), dtype=int32)
Tensor("stack_25:0", shape=(4, 4, 4), dtype=int32)
Tensor("Reshape_26:0", shape=(1

In [5]:
import tensorflow as tf
import numpy as np



matrix = np.load('./groundtruth_z//tps_trained.npy') #读入.npy文件
tps_base0 = np.loadtxt(r'A_val_real.txt').astype(np.float32)

a_loss = matrix - tps_base0

aa = tf.constant(a_loss,dtype = 'float32')
print(aa)

bb = tf.transpose(aa,[1,0])
bb = tf.reshape(bb, [16,240,240,1])


with tf.Session() as sess:
    
    filter_ = np.array([[0,1,0],[1,-4,1],[0,1,0]]).reshape(3,3,1,1)###3*3,输入3通道，输出3通道
    
    
    filter_1 = tf.constant(filter_,dtype = 'float32')
    Laplace_img = tf.nn.conv2d(input = bb,filter = filter_1,strides=[1,1,1,1],padding='SAME')
    
    loss_A_smooth = tf.reduce_sum(tf.square(Laplace_img))
    
    dd = sess.run(bb)
    print(matrix.max(), matrix.min())
    
    #print(sess.run(Laplace_img))
    print(sess.run(loss_A_smooth))
    






Tensor("Const_12:0", shape=(57600, 16), dtype=float32)
1.9095305 -0.49275446
0.20306881


In [1]:
import tensorflow as tf
import numpy as np
 
input1 = tf.placeholder(tf.float32,shape = (2,1))
input2 = tf.placeholder(tf.float32)
 
output = tf.multiply(input1[0], input2)
 
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(output, feed_dict = {input1:[[2.0],[3.0]], input2: 3.2}))


/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-

[6.4]
